In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import keras
import glob
import cv2

import numpy as np # linear algebra
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
import shutil



In [ ]:
!ls ../input/resized-2015-2019-blindness-detection-images

In [ ]:
def empty_dir(path):
    folder = path
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:

df = pd.read_csv('../input/resized-2015-2019-blindness-detection-images/labels/trainLabels15.csv')
df.drop(df.loc[df['level']==0].index, inplace=True)


it=df.head(5)
DATADIR2 = '../input/resized-2015-2019-blindness-detection-images/resized train 15/'
for _,row in it.iterrows():
    print (row['image'],' ',row['level'])   
    path = os.path.join(DATADIR2,row['image']+".jpg")
image = load_img(path)
print(path)
plt.imshow(image)
    

In [ ]:
data = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
print(data.head())

image = load_img("../input/aptos2019-blindness-detection/train_images/44e951e45dca.png")




In [ ]:

dfTrain2015 = pd.read_csv('../input/resized-2015-2019-blindness-detection-images/labels/trainLabels15.csv')
dfTrain2015.drop(dfTrain2015.loc[dfTrain2015['level']==0].index, inplace=True)

dfTest2015 = pd.read_csv('../input/resized-2015-2019-blindness-detection-images/labels/testLabels15.csv')
dfTest2015.drop(dfTest2015.loc[dfTest2015['level']==0].index, inplace=True)


In [ ]:
data.hist(bins=50,figsize=(10,5))
plt.show()

In [ ]:
dfTrain2015.hist(bins=50,figsize=(10,5))
plt.show()

# **Cropping functions**

In [ ]:
import os
import glob
import cv2
import numpy as np

def crop_image_from_gray(img,tol=7):
    """
    Crop out black borders
    https://www.kaggle.com/ratthachat/aptos-updated-preprocessing-ben-s-cropping
    """  
    
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        mask = gray_img>tol        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0):
            return img
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img

def circle_crop_v2(img):
    """
    Create circular crop around image centre
    """
    img = crop_image_from_gray(img)

    height, width, depth = img.shape
    largest_side = np.max((height, width))
    img = cv2.resize(img, (largest_side, largest_side))

    height, width, depth = img.shape

    x = int(width / 2)
    y = int(height / 2)
    r = np.amin((x, y))

    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x, y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)

    return img

#img = cv2.imread("../input/aptos2019-blindness-detection/train_images/0104b032c141.png")
#crop_image_from_gray(img)
#plt.imshow(img)


# Ben Graham's Preprocessing Function

In [ ]:
def benGraham(img):
    image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    IMG_SIZE = 512
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , IMG_SIZE/10) ,-4 ,128) # the trick is to add this line
    return image


In [ ]:
from matplotlib import pyplot as plt
fig, ax3 = plt.subplots(nrows=1, ncols=4, figsize=(20,4))

spaceboy = cv2.imread('../input/aptos2019-blindness-detection/train_images/1df0a4c23c95.png')
ax3[0].imshow(cv2.cvtColor(spaceboy, cv2.COLOR_BGR2RGB))
ax3[0].axis('off')


cropboy = cv2.imread('../input/aptos2019-blindness-detection/train_images/0a1076183736.png')
ax3[1].imshow(cv2.cvtColor(cropboy, cv2.COLOR_BGR2RGB))
ax3[1].axis('off')

squareboy = cv2.imread('../input/aptos2019-blindness-detection/train_images/0e3572b5884a.png')
ax3[2].imshow(cv2.cvtColor(squareboy, cv2.COLOR_BGR2RGB))
ax3[2].axis('off')

supercropboy = cv2.imread('../input/aptos2019-blindness-detection/train_images/698d6e422a80.png')
ax3[3].imshow(cv2.cvtColor(supercropboy, cv2.COLOR_BGR2RGB))
ax3[3].axis('off')
###################################
fig, ax0 = plt.subplots(nrows=1, ncols=4, figsize=(20,4))

img = cv2.imread("../input/aptos2019-blindness-detection/train_images/1df0a4c23c95.png")

spaceboy = crop_image_from_gray(img)
ax0[0].imshow(cv2.cvtColor(spaceboy, cv2.COLOR_BGR2RGB))
ax0[0].axis('off')

img = cv2.imread("../input/aptos2019-blindness-detection/train_images/0a1076183736.png")

cropboy = crop_image_from_gray(img)
ax0[1].imshow(cv2.cvtColor(cropboy, cv2.COLOR_BGR2RGB))
ax0[1].axis('off')

img = cv2.imread("../input/aptos2019-blindness-detection/train_images/0e3572b5884a.png")

squareboy = crop_image_from_gray(img)
ax0[2].imshow(cv2.cvtColor(squareboy, cv2.COLOR_BGR2RGB))
ax0[2].axis('off')

img = cv2.imread("../input/aptos2019-blindness-detection/train_images/698d6e422a80.png")

supercropboy = crop_image_from_gray(img)
ax0[3].imshow(cv2.cvtColor(supercropboy, cv2.COLOR_BGR2RGB))
ax0[3].axis('off')

######################################################
fig, ax2 = plt.subplots(nrows=1, ncols=4, figsize=(20,4))
spaceboy = cv2.imread('../input/aptos2019-blindness-detection/train_images/1df0a4c23c95.png')

spaceboy = circle_crop_v2(img)
ax2[0].imshow(cv2.cvtColor(spaceboy, cv2.COLOR_BGR2RGB))
ax2[0].axis('off')
img = cv2.imread("../input/aptos2019-blindness-detection/train_images/0a1076183736.png")

cropboy = circle_crop_v2(img)
ax2[1].imshow(cv2.cvtColor(cropboy, cv2.COLOR_BGR2RGB))
ax2[1].axis('off')
img = cv2.imread("../input/aptos2019-blindness-detection/train_images/0e3572b5884a.png")

squareboy = circle_crop_v2(img)
ax2[2].imshow(cv2.cvtColor(squareboy, cv2.COLOR_BGR2RGB))
ax2[2].axis('off')
img = cv2.imread("../input/aptos2019-blindness-detection/train_images/698d6e422a80.png")

supercropboy = circle_crop_v2(img)
ax2[3].imshow(cv2.cvtColor(supercropboy, cv2.COLOR_BGR2RGB))
ax2[3].axis('off')
#####################################################
fig, ax4 = plt.subplots(nrows=1, ncols=4, figsize=(20,4))

spaceboy = benGraham(spaceboy)
ax4[0].imshow(cv2.cvtColor(spaceboy, cv2.COLOR_BGR2RGB))
ax4[0].axis('off')

cropboy = benGraham(cropboy)
ax4[1].imshow(cv2.cvtColor(cropboy, cv2.COLOR_BGR2RGB))
ax4[1].axis('off')

squareboy = benGraham(squareboy)
ax4[2].imshow(cv2.cvtColor(squareboy, cv2.COLOR_BGR2RGB))
ax4[2].axis('off')

supercropboy = benGraham(supercropboy)
ax4[3].imshow(cv2.cvtColor(supercropboy, cv2.COLOR_BGR2RGB))
ax4[3].axis('off')

# Combiation of Cropping and Ben Garham functions

In [ ]:
def processingWithCircleCrop(data,beforePath,afterPath,ext):
    for i in data:
        path=os.path.join(beforePath+i+ext)
        img=cv2.imread(path)
        img=circle_crop_v2(img)
        img = benGraham(img)
        outImg=Image.fromarray(img)
        outImg.save(os.path.join(afterPath,i+'.png'))

In [ ]:

def process(data,beforePath,afterPath,ext):
    processingWithCircleCrop(data,beforePath,afterPath,ext)
    print("Process completed")

# Preparing the data

In [ ]:
!mkdir '/kaggle/working/train.zip/'
    
AfterPath='/kaggle/working/trainData2015/'
if(os.path.exists(AfterPath)== False):
    !mkdir '/kaggle/working/trainData2015/'
    
    
AfterPath='/kaggle/working/testData2019/'

if(os.path.exists(AfterPath)== False):
    !mkdir '/kaggle/working/testData2019/'
    
AfterPath='/kaggle/working/testData2015/'

if(os.path.exists(AfterPath)== False):
    !mkdir '/kaggle/working/testData2015/'
    

In [ ]:
trainData2019BeforePath='../input/aptos2019-blindness-detection/train_images/'
trainData2015BeforePath='../input/resized-2015-2019-blindness-detection-images/resized train 15/'

testData2019BeforePath='../input/aptos2019-blindness-detection/test_images/'  
testData2015BeforePath='../input/resized-2015-2019-blindness-detection-images/resized test 15/'
###
trainData2019AfterPath='/kaggle/working/trainData2019/'
trainData2015AfterPath='/kaggle/working/trainData2015/'

testData2019AfterPath='/kaggle/working/testData2019/'    
testData2015AfterPath='/kaggle/working/testData2015/'
###
dfTrain2015 = pd.read_csv('../input/resized-2015-2019-blindness-detection-images/labels/trainLabels15.csv')
dfTrain2015.drop(dfTrain2015.loc[dfTrain2015['level']==0].index, inplace=True)

dfTrain2019 = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')

####

trainData2015=dfTrain2015['image']


trainData2019=dfTrain2019['id_code']


# Combining 2015 and 2019 Data for a more balanced Data destribution 

In [ ]:
def setIndx(lvl):
    lvl.set_index(["image"], inplace = True, 
                    append = True, drop = True) 
   
    # resetting index 

    lvl.reset_index(inplace = True) 

In [ ]:
df = dfTrain2015

lvl1 =df.loc[df['level'] == 1]
lvl2 =df.loc[df['level'] == 2]
lvl3 =df.loc[df['level'] == 3]
lvl4 =df.loc[df['level'] == 4]
setIndx(lvl1)
setIndx(lvl2)
setIndx(lvl3)
setIndx(lvl4)

print("##################")
print(lvl1.head())
print("##################")
print(lvl2.tail())
print("##################")
print(lvl3.head())
print("##################")
print(lvl4.head())
print(lvl1.shape,lvl2.shape)

In [ ]:
lvl1 = lvl1.drop(lvl1[lvl1.index > 1299].index) 
lvl2 = lvl2.drop(lvl2[lvl2.index > 699].index) 
print(lvl1.shape,lvl2.shape,lvl3.shape,lvl4.shape)

In [ ]:

frames = [lvl1, lvl2, lvl3,lvl4]
final2015df = pd.concat(frames)
final2015df

In [ ]:

from sklearn.utils import shuffle
final2015df=shuffle(final2015df)
final2015df = final2015df.drop('level_0', 1)
setIndx(final2015df)
final2015df = final2015df.drop('level_0', 1)
final2015df

In [ ]:
final2015df.rename({'image': 'id_code', 'level': 'diagnosis'}, axis=1, inplace=True)
frames = [final2015df,dfTrain2019]
fulldf = pd.concat(frames)
fulldf.shape

# Difference of Data destribution between the original dataset and the combined dataset:

old data:

In [ ]:
data.hist(bins=50,figsize=(10,5))
plt.show()

combined data:

In [ ]:
fulldf.hist(bins=50,figsize=(10,5))
plt.show()

In [ ]:
final2015df.head()

# Preprocessing The data 

In [ ]:
trainData2015=final2015df['id_code']
trainData2019=dfTrain2019['id_code']

In [ ]:
process(trainData2019,trainData2019BeforePath,trainData2019AfterPath,'.png')
process(trainData201,trainData2015BeforePath,trainData2015AfterPath,'.jpg')


In [ ]:
import zipfile

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

zipf = zipfile.ZipFile('/kaggle/working/train.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir(trainData2015AfterPath, zipf)
zipdir(trainData2019AfterPath, zipf)

zipf.close()
